In [ ]:
import os
import time
import config
from datetime import datetime
from log_utils import read_log_messages, normalize_template, normalize_template_v1, normalize_template_v2, save_templates
from debt_utils import get_code_snippets, get_td_ground_truth, save_td_labels, normalize_td_label
from ollama_utils import start_ollama_server, stop_ollama_server, start_ollama_server_log
from agent_utils import create_agent, save_agent_responses
from codecarbon import EmissionsTracker, OfflineEmissionsTracker
from evaluation import evaluate_and_save_parsing, evaluate_and_save_td

In [ ]:
# --- Configuration ---
llm_config = config.LLM_CONFIG

DATA_DIR = config.DATA_DIR
RESULT_DIR = config.RESULT_DIR
os.makedirs(RESULT_DIR, exist_ok=True)
 
TASK = (config.TASK).capitalize()
DESIGN = (config.DESIGN).capitalize()
project_name = f"{TASK}_{DESIGN}"
model = llm_config["config_list"][0]["model"].replace(":", "-")
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name = f"{project_name}_{model}_{timestamp}"

input_file = config.IN_FILE
input_file_path = os.path.join(DATA_DIR, input_file)
ground_truth_file = config.GT_FILE
ground_truth_file_path = os.path.join(DATA_DIR, ground_truth_file)



TD Detection

In [ ]:
# --- Task-specific configuration for TD detection ---
task_prompt = config.TASK_PROMPT_TD_DETECTION
sys_prompt_generator = config.SYS_MSG_TD_DETECTION_GENERATOR_FEW_SHOT

# --- Read Input Data ---
code_snippets = get_code_snippets(input_file_path)


In [ ]:
# --- With CodeCarbon Emissions Tracking for Technical Debt Detection Task---
def run_inference_with_emissions_td_detection(code_snippets, llm_config, sys_prompt, task_prompt, exp_name, result_dir):
    td_results = []
    tracker = OfflineEmissionsTracker(project_name=exp_name, output_dir=result_dir, country_iso_code="CAN", save_to_file=True)
    tracker.start()
    try:
        td_detector_gen = create_agent(
            agent_type="assistant",
            name="td_detection_generator_agent",
            llm_config=llm_config,
            sys_prompt=sys_prompt,
            description="Analyze the code snippet in order to determine whether it contains a code smell."
        )
        for i, code_snippet in enumerate(code_snippets):
            content = task_prompt + code_snippet
            #print(f"Processing code_snippet {i+1}/{len(code_snippets)}")
            res = td_detector_gen.generate_reply(messages=[{"content": content, "role": "user"}])
            if res is not None and "content" in res:
                td_results.append(res["content"].strip())
                #print(f"[Info] Processed code_snippet {i}: {res['content'].strip()}")
            else:
                td_results.append("NONE")
                print(f"[Warning] Skipped code_snippet {i} — no response or invalid format.")
    finally:
        emissions = tracker.stop()
    print(f"Emissions: {emissions} kg CO2")
    return td_results


In [ ]:
# --- Main Execution ---
proc = start_ollama_server()
time.sleep(5)  # Give it some time to initialize
td_results = run_inference_with_emissions_td_detection(code_snippets, llm_config, sys_prompt_generator, task_prompt, exp_name, RESULT_DIR)
save_td_labels(td_results, llm_config, DESIGN, RESULT_DIR)
stop_ollama_server(proc)

In [ ]:
# Load ground truth
gt = get_td_ground_truth(ground_truth_file_path)
# Evaluate and save results
results = evaluate_and_save_td(normalize_td_label, gt, td_results, exp_name)

Log Parsing

In [ ]:
# --- Task-specific configuration for Log parsing ---
task_prompt = config.TASK_PROMPT_LOG_PARSING
sys_prompt_generator = config.SYS_MSG_SINGLE_LOG_PARSER_FEW_SHOT

# --- Read Input Data ---
logs = read_log_messages(input_file_path)

In [ ]:
# --- With CodeCarbon Emissions Tracking for Log Parsing Task---
def run_inference_with_emissions_log_parsing(logs, llm_config, sys_prompt_log_parser, task_prompt, exp_name, result_dir):
    parsed_templates = []
    tracker = OfflineEmissionsTracker(project_name=exp_name, output_dir=result_dir, country_iso_code="CAN", save_to_file=True)
    tracker.start()
    try:
        log_parser = create_agent(
            agent_type="assistant",
            name="log_parser_agent",
            llm_config=llm_config,
            sys_prompt=sys_prompt_log_parser,
            description="Analyze the log message in order to determine the corresponding template."
        )
        for i, log_message in enumerate(logs):
            content = task_prompt + log_message
            res = log_parser.generate_reply(messages=[{"content": content, "role": "user"}])
            if res is not None and "content" in res:
                parsed_templates.append(res["content"].strip())
                #print(f"[Info] Processed log {i}: {res['content'].strip()}")
            else:
                parsed_templates.append("NONE")
                print(f"[Warning] Skipped log {i} — no response or invalid format.")
    finally:
        emissions = tracker.stop()
    print(f"Emissions: {emissions} kg CO2")
    return parsed_templates


In [ ]:
# --- Main Execution ---
proc = start_ollama_server()
time.sleep(5)  # Give it some time to initialize
parsed_templates = run_inference_with_emissions_log_parsing(logs, llm_config, sys_prompt_generator, task_prompt, exp_name, RESULT_DIR)
save_templates(parsed_templates, llm_config, DESIGN, RESULT_DIR)
stop_ollama_server(proc)

In [ ]:
results = evaluate_and_save_parsing(normalize_template, parsed_templates, ground_truth_file_path, exp_name)
results_v1 = evaluate_and_save_parsing(normalize_template_v1, parsed_templates, ground_truth_file_path, exp_name)
results_v2 = evaluate_and_save_parsing(normalize_template_v2, parsed_templates, ground_truth_file_path, exp_name)

In [ ]:
#print("Results:", results)